In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 목표
- mlp, cnn 모델을 활용하여 개, 고양이 이진 클래스 분류하는 모델을 만들어보자!
- 모델의 성능을 개선시켜보자!
- 데이터 증식, 기존의 잘 학습된 모델을 사용하는 방법(전이학습)

#### 이미지 데이터 전처리
- 사진을 데이터화 시키는 작업
- 이미지 -> 배열
- 압축된 형식의 배열로 저장 : npz

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from zipfile import ZipFile # 압축 관련 모듈

In [5]:
%cd /content/drive/MyDrive/Colab Notebooks/DeepLearning_빅데16

/content/drive/MyDrive/Colab Notebooks/DeepLearning_빅데16


In [10]:
# zip 파일 압축 해제하여 이미지를 불러오기
zip_file = "./data/cats_and_dogs_filtered.zip"

# 파일 다루는 파이썬 함수 with open(경로, 모드설정)
with ZipFile(zip_file,'r') as z:
    #압축 파일의 내용 확인하기
    # z.printdir() 경로 확인

    # 압축파일 해제하기
    z.extractall(path="./data/")


In [14]:
# 이미지 접근하려면 경로가 필요 ./data/cats_and_dogs_filtered/train/cats/cat.1.jpg

train_cats_dir="./data/cats_and_dogs_filtered/train/cats"
train_dogs_dir="./data/cats_and_dogs_filtered/train/dogs"

test_cats_dir="./data/cats_and_dogs_filtered/test/cats"
test_dogs_dir="./data/cats_and_dogs_filtered/test/dogs"

# os.listdir(): 해당 경로에 있는 파일명들을 리스트에 순서대로 저장

train_cats_fnames = os.listdir(train_cats_dir)
train_dogs_fnames = os.listdir(train_dogs_dir)

test_cats_fnames = os.listdir(test_cats_dir)
test_dogs_fnames = os.listdir(test_dogs_dir)


In [26]:
print(len(train_cats_fnames))
print(len(train_dogs_fnames))
print(len(test_cats_fnames))
print(len(test_dogs_fnames))

1000
1000
500
500


In [20]:
test_dogs_fnames[:5]

# 경로 연결하는 방법
print(test_dogs_dir +"/"+ test_dogs_fnames[0])
print(os.path.join(test_dogs_dir,test_dogs_fnames[0]))
test_path = os.path.join(test_dogs_dir,test_dogs_fnames[0])

./data/cats_and_dogs_filtered/test/dogs/dog.2000.jpg
./data/cats_and_dogs_filtered/test/dogs/dog.2000.jpg


In [25]:
# 이미지 불러와서 사이즈 조정 및 배열로 변환
# (224,224) 사이즈 변경
img_size = (224,224)
img = Image.open(test_path).resize(img_size)
np.array(img)

array([[[ 74, 103,  37],
        [ 81, 110,  44],
        [107, 136,  69],
        ...,
        [146, 151, 119],
        [188, 193, 161],
        [168, 174, 139]],

       [[ 93, 123,  55],
        [ 82, 112,  45],
        [ 84, 114,  46],
        ...,
        [138, 143, 113],
        [186, 191, 160],
        [180, 186, 152]],

       [[116, 147,  75],
        [105, 136,  65],
        [ 95, 126,  56],
        ...,
        [150, 155, 125],
        [217, 222, 190],
        [239, 245, 211]],

       ...,

       [[134, 174,  81],
        [143, 182,  88],
        [134, 174,  79],
        ...,
        [ 98, 142,  55],
        [ 89, 133,  47],
        [ 86, 132,  45]],

       [[171, 213, 124],
        [168, 211, 121],
        [161, 204, 114],
        ...,
        [111, 153,  76],
        [ 93, 135,  59],
        [107, 149,  72]],

       [[179, 222, 133],
        [181, 224, 136],
        [182, 225, 136],
        ...,
        [105, 147,  71],
        [ 95, 137,  61],
        [127, 169,  93]]

In [32]:
# 사진을 불러와서 배열로 변경하는 함수 정의
# 사진 크기는 전부 224,224

def load_images(folder_path, file_names,image_size=(224,224)):
    image = []
    for i in file_names: # i : 파일 이름 하나씩 접근

    # 폴더 경로 + 파일명 합치기
        path = os.path.join(folder_path,i)

    # 파일 오픈, 크기 조정
        img = Image.open(path).resize(image_size)

    # 배열로 변경, image 빈 리스트에 저장
        image.append(np.array(img))
    return np.array(image)

In [31]:
# 함수 호출 -> 4개의 각 폴더의 있는 이미지를 전처리

X_train_cats = load_images(train_cats_dir,train_cats_fnames)
X_train_cats.shape

(1000, 224, 224, 3)

In [33]:
# 함수 호출 -> 4개의 각 폴더의 있는 이미지를 전처리

X_train_dogs = load_images(train_dogs_dir,train_dogs_fnames)
X_test_cats = load_images(test_cats_dir,test_cats_fnames)
X_test_dogs = load_images(test_dogs_dir,test_dogs_fnames)

print(X_train_dogs.shape)
print(X_test_cats.shape)
print(X_test_dogs.shape)


(1000, 224, 224, 3)
(500, 224, 224, 3)
(500, 224, 224, 3)


In [35]:
# train끼리 병합
# test 병합
# 고양이 + 개 이미지 순서대로 병합

X_train = np.concatenate((X_train_cats,X_train_dogs))
X_test = np.concatenate((X_test_cats,X_test_dogs))
print(X_train.shape, X_test.shape)

(2000, 224, 224, 3) (1000, 224, 224, 3)


In [39]:
# y_train, y_test
# 고양이 : 0, 강아지 : 1
# 정답데이터 만들기

y_train = np.array([0]*1000 + [1]*1000)
y_test = np.array([0]*500 + [1]*500)
print(y_train.shape, y_test.shape)

(2000,) (1000,)


In [40]:
# npz 넘파이 배열 압축 형식으로 데이터 저장
# 폴더 경로와 파일명 설정
np.savez_compressed('./data/cats_and_dogs.npz',
                    X_train = X_train, #훈련 문제
                    X_test = X_test, #테스트 문제
                    y_train = y_train, #훈련 답
                    y_test = y_test) #테스트 답

#### npz 파일 로딩

In [43]:
data = np.load('./data/cats_and_dogs.npz')
len(data)

4

In [45]:
X_train = data['X_train']
X_test = data['X_test']

y_train = data['y_train']
y_test = data['y_test']

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2000, 224, 224, 3) (2000,)
(1000, 224, 224, 3) (1000,)


#### MLP 모델 생성

In [46]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.callbacks import EarlyStopping

In [86]:
# mlp모델 자유롭게 생성해보기

model_mlp = Sequential()

# 입력층
model_mlp.add(Flatten(input_shape=(224,224,3)))

# 중간층
model_mlp.add(Dense(units=512, activation='relu'))
model_mlp.add(Dense(units=256, activation='relu'))
model_mlp.add(Dense(units=128, activation='relu'))
model_mlp.add(Dense(units=64, activation='relu'))
model_mlp.add(Dense(units=32, activation='relu'))
model_mlp.add(Dense(units=16, activation='relu'))
# 출력층
model_mlp.add(Dense(units=1, activation='sigmoid'))

model_mlp.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_13 (Flatten)        (None, 150528)            0         
                                                                 
 dense_30 (Dense)            (None, 512)               77070848  
                                                                 
 dense_31 (Dense)            (None, 256)               131328    
                                                                 
 dense_32 (Dense)            (None, 128)               32896     
                                                                 
 dense_33 (Dense)            (None, 64)                8256      
                                                                 
 dense_34 (Dense)            (None, 32)                2080      
                                                                 
 dense_35 (Dense)            (None, 16)              

In [88]:
# 학습 / 평가 방법 설정
model_mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])


In [89]:
# 조기중단
f_ealry = EarlyStopping(
    monitor='val_accuracy',
    patience=5
)

# 학습
h1 = model_mlp.fit(X_train,y_train,validation_split=0.3, epochs=100,
                   batch_size = 64,
                   callbacks=[f_ealry])

Epoch 1/100
22/22 [==============================] - 4s 46ms/step - loss: 1615.1255 - accuracy: 0.5400 - val_loss: 4106.8940 - val_accuracy: 0.0000e+00
Epoch 2/100
22/22 [==============================] - 1s 25ms/step - loss: 653.6313 - accuracy: 0.6079 - val_loss: 877.3724 - val_accuracy: 0.0000e+00
Epoch 3/100
22/22 [==============================] - 1s 25ms/step - loss: 615.9830 - accuracy: 0.5993 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/100
22/22 [==============================] - 1s 25ms/step - loss: 998.7578 - accuracy: 0.5979 - val_loss: 719.3415 - val_accuracy: 0.0000e+00
Epoch 5/100
22/22 [==============================] - 1s 25ms/step - loss: 541.4276 - accuracy: 0.5786 - val_loss: 1863.5875 - val_accuracy: 0.0000e+00
Epoch 6/100
22/22 [==============================] - 1s 26ms/step - loss: 287.7567 - accuracy: 0.5893 - val_loss: 16.5787 - val_accuracy: 0.8150
Epoch 7/100
22/22 [==============================] - 1s 29ms/step - loss: 139.4453 - accuracy: 0.6050 - 

In [85]:
# 평가
model_mlp.evaluate(X_test,y_test)

32/32 [==============================] - 0s 6ms/step - loss: 8.4375 - accuracy: 0.5190


[8.437480926513672, 0.5189999938011169]

#### CNN 모델링

In [96]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout,InputLayer
from tensorflow.keras import optimizers

In [103]:
model_cnn = Sequential()

model_cnn.add(InputLayer(input_shape=(224,224,3)))

model_cnn.add(Conv2D(32,(5,5), padding='valid', activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2,2)))

model_cnn.add(Conv2D(64,(5,5), padding='valid', activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2,2)))

model_cnn.add(Flatten())


model_cnn.add(Dense(100, activation='relu'))
model_cnn.add(Dense(25, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.summary()




Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 220, 220, 32)      2432      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 110, 110, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 106, 106, 64)      51264     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 53, 53, 64)       0         
 g2D)                                                            
                                                                 
 flatten_20 (Flatten)        (None, 179776)            0         
                                                                 
 dense_40 (Dense)            (None, 100)             